In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
def load_dataset(dsname):
    metadata = pd.read_csv(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/metadata_final.csv")
    x = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_data0.p", "rb" ))
    y = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_labels.p", "rb" ))
    return metadata, x, y

In [3]:
from sklearn.preprocessing import StandardScaler

def standardize(data):
    scaler = StandardScaler()
    n = data.shape
    return scaler.fit_transform(data.reshape(n[0],-1)).reshape(n)

In [4]:
datasets = ["emodb", "emovo", "ravdess"]
data = {}
for d in datasets:
    md, x, y = load_dataset(d)
    data[d] = {}
    data[d]["x"] = standardize(x)
    data[d]["y"] = y
    data[d]["metadata"] = md

# Define the hypermodel

In [46]:
import tensorflow as tf
import tensorflow.keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, LSTM, TimeDistributed, InputLayer, Reshape, BatchNormalization, Bidirectional

In [31]:
def evaluate_model(model, X_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

In [54]:
def get_m(hp):
    # Define hyper model architecture
    m = Sequential([
        InputLayer(input_shape=(261,128,1)),
        Reshape((9,29,128,1)),
        TimeDistributed(Conv2D(16, kernel_size=(5,5), activation='relu')),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(MaxPooling2D(pool_size=(4,4), strides=2)),

        TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu')),
        TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=2)),

        TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu')),
        TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=1)),
        TimeDistributed(Flatten()),
        
        # Tune dropout layer with values in 0 - 0.6 with stepsize of 0.3
        Dropout(hp.Float("dropout", min_value=0, max_value=0.6, step=0.3)),
        
        # Tune number of units (2 - 3 - 4 - 5) and dropout (alues in 0 - 0.4 with stepsize of 0.2)
        Bidirectional(LSTM(hp.Int("lstm_units", min_value=2, max_value=5, step=1),
                           dropout=hp.Float("lstm_dropout", min_value=0, max_value=0.4, step=0.2),
                           return_sequences=False)),
        Dense(5, activation='softmax')
    ])
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss='categorical_crossentropy',
              metrics=["accuracy"])
    return m

In [76]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        # Define hyper model architecture
        m = Sequential([
            InputLayer(input_shape=(261,128,1)),
            Reshape((9,29,128,1)),
            TimeDistributed(Conv2D(16, kernel_size=(5,5), activation='relu')),
            TimeDistributed(BatchNormalization()),
            TimeDistributed(MaxPooling2D(pool_size=(4,4), strides=2)),

            TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu')),
            TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=2)),

            TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu')),
            TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=1)),
            TimeDistributed(Flatten()),

            # Tune dropout layer with values in 0 - 0.6 with stepsize of 0.3
            Dropout(hp.Float("dropout", min_value=0, max_value=0.6, step=0.3)),

            # Tune number of units (2 - 3 - 4 - 5) and dropout (alues in 0 - 0.4 with stepsize of 0.2)
            Bidirectional(LSTM(hp.Int("lstm_units", min_value=2, max_value=5, step=1),
                               dropout=hp.Float("lstm_dropout", min_value=0, max_value=0.4, step=0.2),
                               return_sequences=False)),
            Dense(5, activation='softmax')
        ])

        # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
        hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

        # Define optimizer, loss, and metrics
        m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=["accuracy"])
        return m

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", values=[8, 16, 32, 64, 128])
            **kwargs,
        )

In [71]:
class MyTuner(kt.tuners.Hyperband):
      def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size", values=[8, 16, 32, 64, 128])
        return super(MyTuner, self).run_trial(trial, *args, **kwargs)# Uses same arguments as the BayesianOptimization Tuner.

# Hyperparameter optimization

Get train/test data

In [22]:
from sklearn.model_selection import train_test_split

train = {}
test = {}
for d in datasets:
    train[d] = {}
    test[d] = {}
    train_df, test_df = train_test_split(data[d]['metadata'], 
                                       test_size = 0.15, 
                                       random_state = 1938,
                                       stratify = data[d]['metadata']['label'])
    train_index = train_df.index
    test_index = test_df.index
    train[d]['x'] = data[d]['x'][train_index]
    train[d]['y'] = data[d]['y'][train_index]
    test[d]['x']= data[d]['x'][test_index]
    test[d]['y']= data[d]['y'][test_index]
    

In [23]:
data['emodb']['y'].shape, train['emodb']['y'].shape, test['emodb']['y'].shape

((3745, 5), (3183, 5), (562, 5))

Select Hyperband tuner and initalization

In [68]:
tuner = kt.Hyperband(get_m,
                     objective="val_loss",
                     max_epochs=100,
                     # total number of trials to run during the search with different hyperparameter values
                     # max_trials=3,
                     # number of models that should be built and fit for each trial with the same hyperparameter values
                     executions_per_trial=2,
                     # the reduction factor for the number of epochs and number of models for each bracket
                     factor=3,
                     # the number of times to iterate over the full Hyperband algorithm
                     hyperband_iterations=10,
                     overwrite=True,
                     directory="/datasets/nicolas_facchinetti/kt_dir",
                     project_name="cnnlstm")
tuner.search_space_summary()

Search space summary
Default search space size: 4
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.6, 'step': 0.3, 'sampling': None}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
lstm_dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.4, 'step': 0.2, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [77]:
# instantiate the tuner
tuner = kt.Hyperband(MyHyperModel,
                     objective="val_loss",
                     max_epochs=100,
                     # total number of trials to run during the search with different hyperparameter values
                     # max_trials=3,
                     # number of models that should be built and fit for each trial with the same hyperparameter values
                     executions_per_trial=2,
                     # the reduction factor for the number of epochs and number of models for each bracket
                     factor=3,
                     # the number of times to iterate over the full Hyperband algorithm
                     hyperband_iterations=10,
                     overwrite=True,
                     directory="/datasets/nicolas_facchinetti/kt_dir",
                     project_name="cnnlstm")
tuner.search_space_summary()

Search space summary
Default search space size: 0


In [85]:
# instantiate the tuner
tuner = MyTuner(get_m,
                objective="val_loss",
                max_epochs=1000,
                # total number of trials to run during the search with different hyperparameter values
                # max_trials=3,
                # number of models that should be built and fit for each trial with the same hyperparameter values
                executions_per_trial=2,
                # the reduction factor for the number of epochs and number of models for each bracket
                factor=2,
                # the number of times to iterate over the full Hyperband algorithm
                hyperband_iterations=10,
                overwrite=True,
                directory="/datasets/nicolas_facchinetti/kt_dir",
                project_name="cnnlstm")
tuner.search_space_summary()

Search space summary
Default search space size: 4
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.6, 'step': 0.3, 'sampling': None}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
lstm_dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.4, 'step': 0.2, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [86]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=0, min_delta=1e-4, mode='min')

tuner.search(train['emodb']['x'], train['emodb']['y'],
             epochs=100,
             validation_split=0.2,
             callbacks=[earlyStopping, reduce_lr_loss],
             verbose=2)

Trial 353 Complete [00h 00m 12s]
val_loss: 1.6034173369407654

Best val_loss So Far: 1.1538488864898682
Total elapsed time: 01h 20m 09s
INFO:tensorflow:Oracle triggered exit


In [95]:
tuner.results_summary()

Results summary
Results in /datasets/nicolas_facchinetti/kt_dir/cnnlstm
Showing 10 best trials
Trial summary
Hyperparameters:
dropout: 0.0
lstm_units: 4
lstm_dropout: 0.2
learning_rate: 0.0001
batch_size: 8
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 9
tuner/round: 0
Score: 1.1538488864898682
Trial summary
Hyperparameters:
dropout: 0.3
lstm_units: 4
lstm_dropout: 0.4
learning_rate: 0.0001
batch_size: 8
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 9
tuner/round: 0
Score: 1.202318549156189
Trial summary
Hyperparameters:
dropout: 0.0
lstm_units: 3
lstm_dropout: 0.2
learning_rate: 0.0001
batch_size: 8
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 9
tuner/round: 0
Score: 1.2051946520805359
Trial summary
Hyperparameters:
dropout: 0.0
lstm_units: 4
lstm_dropout: 0.4
learning_rate: 0.001
batch_size: 16
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 9
tuner/round: 0
Score: 1.2085976600646973
Trial summary
Hyperparameters:
dropout: 0.0
lstm_units: 5
lstm_dropout

In [96]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

# Build model
h_model = tuner.hypermodel.build(best_hps)

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_delta=1e-4, mode='min')

# Train the hypertuned model
h_model.fit(train['emodb']['x'], train['emodb']['y'], epochs=100, validation_split=0.2, callbacks=[earlyStopping, reduce_lr_loss], verbose=2)

Epoch 1/100
80/80 - 3s - loss: 1.5904 - accuracy: 0.2675 - val_loss: 1.5720 - val_accuracy: 0.3407
Epoch 2/100
80/80 - 1s - loss: 1.4821 - accuracy: 0.3653 - val_loss: 1.4589 - val_accuracy: 0.4176
Epoch 3/100
80/80 - 1s - loss: 1.3664 - accuracy: 0.4462 - val_loss: 1.3781 - val_accuracy: 0.4615
Epoch 4/100
80/80 - 1s - loss: 1.2855 - accuracy: 0.4973 - val_loss: 1.2818 - val_accuracy: 0.4961
Epoch 5/100
80/80 - 1s - loss: 1.2345 - accuracy: 0.5169 - val_loss: 1.2220 - val_accuracy: 0.5338
Epoch 6/100
80/80 - 1s - loss: 1.1871 - accuracy: 0.5440 - val_loss: 1.1745 - val_accuracy: 0.5290
Epoch 7/100
80/80 - 1s - loss: 1.1559 - accuracy: 0.5546 - val_loss: 1.1498 - val_accuracy: 0.5306
Epoch 8/100
80/80 - 1s - loss: 1.1242 - accuracy: 0.5829 - val_loss: 1.1043 - val_accuracy: 0.5824
Epoch 9/100
80/80 - 1s - loss: 1.0924 - accuracy: 0.5970 - val_loss: 1.0824 - val_accuracy: 0.6028
Epoch 10/100
80/80 - 1s - loss: 1.0667 - accuracy: 0.6080 - val_loss: 1.0642 - val_accuracy: 0.5777
Epoch 11/

80/80 - 1s - loss: 0.2043 - accuracy: 0.9635 - val_loss: 0.5550 - val_accuracy: 0.7991
Epoch 84/100
80/80 - 1s - loss: 0.2018 - accuracy: 0.9627 - val_loss: 0.5537 - val_accuracy: 0.8053
Epoch 85/100
80/80 - 1s - loss: 0.1925 - accuracy: 0.9682 - val_loss: 0.5471 - val_accuracy: 0.8085
Epoch 86/100
80/80 - 1s - loss: 0.1900 - accuracy: 0.9682 - val_loss: 0.5423 - val_accuracy: 0.8100
Epoch 87/100
80/80 - 1s - loss: 0.1870 - accuracy: 0.9713 - val_loss: 0.5369 - val_accuracy: 0.8100
Epoch 88/100
80/80 - 1s - loss: 0.1746 - accuracy: 0.9737 - val_loss: 0.5498 - val_accuracy: 0.8022
Epoch 89/100
80/80 - 1s - loss: 0.1828 - accuracy: 0.9705 - val_loss: 0.5543 - val_accuracy: 0.8038
Epoch 90/100
80/80 - 1s - loss: 0.1740 - accuracy: 0.9713 - val_loss: 0.5485 - val_accuracy: 0.8132
Epoch 91/100
80/80 - 1s - loss: 0.1727 - accuracy: 0.9705 - val_loss: 0.5553 - val_accuracy: 0.8100
Epoch 92/100
80/80 - 1s - loss: 0.1800 - accuracy: 0.9658 - val_loss: 0.5589 - val_accuracy: 0.7991
Epoch 93/100


In [97]:
results = []
# Evaluate model on test set
hyper_df = evaluate_model(h_model, test['emodb']['x'], test['emodb']['y'])

# Set index to hypertuned
hyper_df.index = ["Hypertuned"]

# Append results in dataframe
results.append(hyper_df)
hyper_df

18/18 [==============================] - 0s 5ms/step - loss: 0.5570 - accuracy: 0.7989


,loss,accuracy
Hypertuned,0.556999,0.798932


In [ ]:

def create_model_opt():
    new_model = tensorflow.keras.Sequential.from_config(chosen)
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model
    
model = KerasClassifier(build_fn=create_model_opt, verbose=0)

In [ ]:
from sklearn.model_selection import GridSearchCV
# define the grid search parameters
batch_size = [16, 32, 64, 128]
epochs = [30, 50, 70]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [ ]:
results_opt = {}
for d in data:
    print(f'Dataset {d}')
    
    x = data[d]["x"]
    y = data[d]["y"]
    metadata = data[d]["metadata"]
    cv = leave_one_speaker_out(metadata, n=3)
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, verbose=3)
    grid_result = grid.fit(x, y)
    results_opt[d] = grid_result
    

In [ ]:
# summarize results
for d in results_opt:
    print(d)
    grid_result = results_opt[d]
    print("\tBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("\t%f (%f) with: %r" % (mean, stdev, param))